In [61]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")
response = llm.invoke([HumanMessage(content="잘 지냈어?")])
print(response.content)

네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요? 도움이 필요하신 부분이 있다면 말씀해 주세요.


In [62]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone:str, location: str) -> str:
    """현재 시각을 반환하는 함수
    
    Args:
        timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야 함
        location (str): 지역명, 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}'
    print(location_and_local_time)
    return location_and_local_time

In [63]:
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

llm_with_tools = llm.bind_tools(tools)

In [64]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.invoke(messages)
messages.append(response);

print(messages);

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JYRvEcfSRsn1vrC9syajOt2m', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 130, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CMuXtnl9AV4M9KPGGG3ycnUeoyDpa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--154f44e4-9d32-4865-b4cc-fe82cac2ea0e-0', tool_call

In [65]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2025-10-04 20:16:59


In [66]:
from pydantic import BaseModel, Field

class StockhistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: Id, 1mo, 1y)")
    

In [67]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockhistoryInput) -> str:
    """주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(stock_history_input.period)
    history_md = history.to_markdown()
    
    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)


In [68]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

content='' additional_kwargs={'tool_calls': [{'id': 'call_woTGHwBylAR0DybJhLPoABio', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 275, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CMuXulVW6xSzuotZJ9D3AcCFXZpyP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ababfe14-e036-471b-845c-d14c18e87b29-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_woTGHwBylAR0DybJhLPoABio', 'type': 'tool_call'}] usage_metadata={'inp